<a href="https://colab.research.google.com/github/yoon777/DACON_GBT_competition/blob/main/KoBERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

In [67]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct  7 07:34:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   77C    P0              35W /  72W |   2903MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [145]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 56.9 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install bertopic
!pip install bertopic[visualization]
!pip install transformers
!pip install sentence_transformers
!pip install konlpy
!pip install optuna

In [53]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.


In [54]:
%cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

In [56]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
from bertopic import BERTopic
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# 데이터 불러오기

In [59]:
train = pd.read_csv('/content/drive/MyDrive/gbt_hackathon/data/train.csv')
train_sample_10000 = pd.read_csv('/content/drive/MyDrive/gbt_hackathon/data/train_sample_10000.csv')
train_sample_10000.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."


# 데이터 수정

In [60]:
# 두 문자열을 합치기
train['text'] = train['제목'] + " " + train['키워드']  # 사이에 공백 추가
train_sample_10000['text'] = train_sample_10000['제목'] + " " + train_sample_10000['키워드']  # 사이에 공백 추가
train_sample_10000['text'].head(5)

,text
0,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최 용인문화재단,인..."
1,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원 용인,농촌,테마파크,단체,체험객..."
2,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장 용인시,노후,주택,에너지,..."
3,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대 수원,용인,고양시,특례시,..."
4,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할 용인시,스페인,미국,국..."


# BERTopic 기본 코드 -> 폐기

https://wikidocs.net/162076

In [63]:
from bertopic import BERTopic
from tqdm import tqdm
import torch

# GPU 사용 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

list_of_text = train_sample_10000['text'].tolist()

# BERTopic 모델 생성
model = BERTopic(verbose=True)
list_of_text_topics, list_of_text_probs = model.fit_transform(list_of_text)

# 결과 출력
print('각 문서의 토픽 번호 리스트:', len(list_of_text_topics))
print('첫번째 문서의 토픽 번호:', list_of_text_topics[0])

# 주제 정보 출력
topic_info = model.get_topic_info()
topic_info

2024-10-07 07:11:45,356 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2024-10-07 07:12:02,229 - BERTopic - Embedding - Completed ✓
2024-10-07 07:12:02,231 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-07 07:12:14,612 - BERTopic - Dimensionality - Completed ✓
2024-10-07 07:12:14,614 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-07 07:12:15,223 - BERTopic - Cluster - Completed ✓
2024-10-07 07:12:15,229 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-07 07:12:15,547 - BERTopic - Representation - Completed ✓


각 문서의 토픽 번호 리스트: 10000
첫번째 문서의 토픽 번호: -1


,Topic,Count,Name,Representation,Representative_Docs
0,-1,6128,-1_10_30_20_11,"[10, 30, 20, 11, 70, 40, 15, 100, 18, 12]","[반도체벨트 최중심 입지 '현대 테라타워 세마역' 기업들 관심 반도체,벨트,입지,현..."
1,0,250,0_tf_tf2827ff3_20242024_474,"[tf, tf2827ff3, 20242024, 474, 240, vs, 100, 4...","[수원시, 특례시 대비 TF팀 구성 수원시,특례,대비,TF,구성,수원시,수원특례시,..."
2,1,192,1_283112102019ictar_mice_77ebs1ebs_3r,"[283112102019ictar, mice, 77ebs1ebs, 3r, 2022,...","[2022년도 환경교육도시 6곳 선정 환경,교육,도시,선정,광역지자체,인천,경기,기..."
3,2,163,2_400_19_2000_300,"[400, 19, 2000, 300, 459, 600, 80, 1000, 635, ...","[경기도 코로나19 확진자 992명 중증환자 병상 가동률 80% 돌파 경기도,코로나..."
4,3,154,3_150147113_172390262262bq283283bq332332bq2062...,"[150147113, 172390262262bq283283bq332332bq2062...","[용인 마평동 종합운동장 부지에 ""지상공원 조성?""vs""터미널 조성?"" 용인,마평동..."
...,...,...,...,...,...
91,90,11,90_26d_353lgrdrd200103rd1arhudarledqdled5165hu...,"[26d, 353lgrdrd200103rd1arhudarledqdled5165hud...","[""무단횡단인데 차 잘못이라고"" 한문철도 '무죄' 기원한 사고 [아차車] 무단횡단,..."
92,91,10,91_31recycleupcycle2731610_33335_482135ktx1323...,"[31recycleupcycle2731610, 33335, 482135ktx1323...","[화성시, ‘서울3호선 연장?경기남부광역철도 사전타당성조사’중간보고회 참석 화성시,..."
93,92,10,92_82003kbnh358kb502pb718787787725fed_89aaa401...,"[82003kbnh358kb502pb718787787725fed, 89aaa4014...","[“1등만 19번 당첨” 로또 명당, 세금으로 길까지 넓힌다? 1등,당첨,로또,명당..."
94,93,10,93_14150_2030lhlh2030761lhds_27ytn_430bsibsibs...,"[14150, 2030lhlh2030761lhds, 27ytn, 430bsibsib...","[용인, 보정동 신라시대 고분군 공개 중부지역 두 번째 규모 용인,보정동,신라,시대..."


In [64]:
model.get_topic(-1)

[('10', 0.007885781479943703),
 ('30', 0.007093020964006296),
 ('20', 0.006805782483275514),
 ('11', 0.006701063814882107),
 ('70', 0.006330333916241139),
 ('40', 0.006306887227262609),
 ('15', 0.006281080969112032),
 ('100', 0.006168555325875219),
 ('18', 0.0061569677849170135),
 ('12', 0.005935779653200682)]

In [65]:
# 토픽 시각화
model.visualize_topics()

# KoBERTopic github 참고

https://github.com/ukairia777/KoBERTopic/blob/main/KoBERTopic.ipynb

## 한국어 학습

In [ ]:
!wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt

--2024-10-07 03:22:03--  https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43694449 (42M) [text/plain]
Saving to: ‘2016-10-20.txt.1’

2016-10-20.txt.1    100%[===================>]  41.67M  --.-KB/s    in 0.1s    

2024-10-07 03:22:03 (408 MB/s) - ‘2016-10-20.txt.1’ saved [43694449/43694449]



In [ ]:
from sklearn.datasets import fetch_20newsgroups

text_file = "2016-10-20.txt"
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
print(docs[:5])

documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

preprocessed_documents = []
for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

print()
print(preprocessed_documents[:5])

["\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n", 'My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:\n\n  - Diamond Stealth Pro Local Bus\n\n  - Orchid Farenheit 1280\n\n  - ATI Graphics Ultra Pro\n\n  - Any other high-perf

100%|██████████| 30091/30091 [00:00<00:00, 152920.51it/s]


['오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 3

In [66]:
## Mecab과 SBERT를 이용한 Bertopic
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

# Mecab을 이용한 토큰화
custom_tokenizer = CustomTokenizer(Mecab())

vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

embedding_model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", device=device)
bertopic = BERTopic(embedding_model=embedding_model,
                    vectorizer_model=vectorizer,
                    nr_topics=50,
                    top_n_words=10,
                    calculate_probabilities=True,
                    verbose=True)

preprocessed_documents_topics, preprocessed_documents_probs = bertopic.fit_transform(preprocessed_documents)

2024-10-07 07:14:14,116 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/861 [00:00<?, ?it/s]

2024-10-07 07:15:45,819 - BERTopic - Embedding - Completed ✓
2024-10-07 07:15:45,821 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-07 07:16:05,214 - BERTopic - Dimensionality - Completed ✓
2024-10-07 07:16:05,217 - BERTopic - Cluster - Start clustering the reduced embeddings


KeyboardInterrupt: 

In [72]:
bertopic.visualize_topics()

TypeError: argument of type 'NoneType' is not iterable

In [73]:
bertopic.visualize_distribution(preprocessed_documents_probs[0])

NameError: name 'preprocessed_documents_probs' is not defined

In [ ]:
bertopic.visualize_topics()

## 데이터에 적용

In [70]:
train_sample_10000.head()

,ID,분류,제목,키워드,text
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...","용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최 용인문화재단,인..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트...","용인 농촌테마파크, 7~8월 단체체험객 체험료 지원 용인,농촌,테마파크,단체,체험객..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화...","용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장 용인시,노후,주택,에너지,..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특...","수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대 수원,용인,고양시,특례시,..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코...","용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할 용인시,스페인,미국,국..."


In [121]:
train_sample_10000.text[0]

'용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최 용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화재단,용인문화재단,제공,시민들,심신,여유,안정,인문학,콘서트,개최,재단,18일,큰어울마당,용인시,평생,학습관,인문학,콘서트,뮤지컬,이야기,인문학,콘서트,방송,매체,왕성,활동,뮤지컬,평론가,교수,원종원,순천향대,진행,이건명,김소향,김보경,유건우,배우들,정상급,뮤지컬,배우,라이브,무대,동시,만끽,공연,공연,홍지민,뮤지컬,배우,참여,라이브,무대,선사,영화,뮤지컬,합성어,무비컬,주제,맘마미아,드림걸즈,유명,작품,교수,위트,해설,예정,공연,초등학생,관람,초등학생,가능,티켓,가격,전석,공연,용인문화재단,누리집,확인'

In [71]:
# 훈련 데이터와 검증 데이터로 분리
from sklearn.model_selection import train_test_split

label_encoder = {label: i for i, label in enumerate(train_sample_10000['분류'].unique())}
train_sample_10000['label'] = train_sample_10000['분류'].map(label_encoder)

train_sample_10000_docs, val_sample_10000_docs = train_test_split(train_sample_10000, test_size=0.2, random_state=42)
train_sample_10000_labels, val_sample_10000_labels = train_test_split(train_sample_10000['label'], test_size=0.2, random_state=42)

In [74]:
train_sample_10000_docs.head()

,ID,분류,제목,키워드,text,label
9254,TRAIN_09254,지역,'용인 플랫폼시티’ 본궤도 경제자족도시 교통허브 도약,"용인,플랫폼,시티,본궤도,경제자족도시,교통허브,도약,275만,밑그림,경제도심,건설,...","'용인 플랫폼시티’ 본궤도 경제자족도시 교통허브 도약 용인,플랫폼,시티,본궤도,경제...",1
1561,TRAIN_01561,지역,"용인특례시의회, 'Sports-City 용인Ⅵ' 인천시 벤치마킹","용인특례시의회,Sports-City,용인,인천시,벤치마킹,용인,위상,체육,도시,경쟁...","용인특례시의회, 'Sports-City 용인Ⅵ' 인천시 벤치마킹 용인특례시의회,Sp...",1
1670,TRAIN_01670,지역,"용인시, 국민권익위원회 청렴도 평가 3등급 ‘추락’","용인시,국민권익위원회,청렴,추락,평가,용인,홍화표,행정권한,확대,특례시,승격,경기,...","용인시, 국민권익위원회 청렴도 평가 3등급 ‘추락’ 용인시,국민권익위원회,청렴,추락...",1
6087,TRAIN_06087,지역,"용인특례시, 기흥저수지 산책로 나뭇잎 쉼터 조성","용인특례시,기흥저수지,산책,조성,나뭇잎,쉼터,강태희,용인특례시,기흥저수지,기흥구,공...","용인특례시, 기흥저수지 산책로 나뭇잎 쉼터 조성 용인특례시,기흥저수지,산책,조성,나...",1
6669,TRAIN_06669,지역,"용인시, 저소득 어르신 가구 ‘잔고장 출장 수리’출동","용인시,저소득,가구,잔고장,출장,수리,출동,아시아투데이,홍화표,용인특례시,일상,불편...","용인시, 저소득 어르신 가구 ‘잔고장 출장 수리’출동 용인시,저소득,가구,잔고장,출...",1


In [83]:
train_sample_10000_labels.unique()

array([ 1, 27, 16, 21, 10, 33, 20, 25, 17,  7,  0, 51, 14, 43, 18, 22, 12,
       28,  5, 11, 15, 46, 26, 29, 32,  6,  3, 36,  8, 54, 39, 31, 35, 30,
       40, 19, 45, 37, 38, 13,  4, 44, 23,  9,  2, 24, 42, 34, 52, 47, 49,
       55, 48, 50, 53, 41])

## KoBERTopic1

In [76]:
# 참고용으로 가져옴(하이퍼 파라미터 확인)
## Mecab과 SBERT를 이용한 Bertopic
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

# Mecab을 이용한 토큰화
custom_tokenizer = CustomTokenizer(Mecab())

vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

embedding_model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", device=device)
kobertopic = BERTopic(embedding_model=embedding_model,
                      vectorizer_model=vectorizer,
                      nr_topics=56,
                      top_n_words=10,
                      calculate_probabilities=True,
                      verbose=True)

In [77]:
kobertopic_train_topics, kobertopic_train_probs = kobertopic.fit_transform(train_sample_10000_docs['text'], y=train_sample_10000_labels)

2024-10-07 07:36:54,347 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/250 [00:00<?, ?it/s]

2024-10-07 07:37:24,480 - BERTopic - Embedding - Completed ✓
2024-10-07 07:37:24,481 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-07 07:37:34,218 - BERTopic - Dimensionality - Completed ✓
2024-10-07 07:37:34,220 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-07 07:37:40,332 - BERTopic - Cluster - Completed ✓
2024-10-07 07:37:40,334 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-07 07:37:46,774 - BERTopic - Representation - Completed ✓
2024-10-07 07:37:46,776 - BERTopic - Topic reduction - Reducing number of topics
2024-10-07 07:37:52,226 - BERTopic - Topic reduction - Reduced number of topics from 118 to 50


In [82]:
kobertopic_train_topics[:5], kobertopic_train_probs

([-1, -1, 0, -1, -1],
 array([[2.02248036e-001, 3.83930937e-001, 9.92967359e-003, ...,
         8.24955675e-003, 5.50757555e-003, 1.46739293e-002],
        [5.66821137e-001, 5.69962952e-002, 1.62522607e-002, ...,
         3.14299932e-003, 2.79970595e-003, 3.95294309e-003],
        [7.65913505e-001, 2.66302721e-002, 1.55900276e-002, ...,
         1.92492660e-003, 1.61767492e-003, 2.43783645e-003],
        ...,
        [7.00739216e-001, 3.77432962e-002, 1.87617701e-002, ...,
         2.94773234e-003, 2.51095675e-003, 3.53544163e-003],
        [3.09664771e-306, 1.75473745e-306, 1.93757447e-307, ...,
         3.37005041e-307, 6.92099677e-307, 2.09796985e-307],
        [7.93144806e-001, 3.10986835e-002, 1.35346766e-002, ...,
         2.21446029e-003, 2.02410135e-003, 2.58026362e-003]]))

In [91]:
kobertopic_val_topics, kobertopic_val_probs = kobertopic.transform(val_sample_10000_docs['text'].tolist())

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2024-10-07 07:43:06,967 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2024-10-07 07:43:25,711 - BERTopic - Dimensionality - Completed ✓
2024-10-07 07:43:25,713 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-10-07 07:43:25,863 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2024-10-07 07:43:27,777 - BERTopic - Probabilities - Completed ✓
2024-10-07 07:43:27,778 - BERTopic - Cluster - Completed ✓


In [93]:
kobertopic_val_topics[:5], kobertopic_val_probs

([-1, -1, 0, -1, -1],
 array([[3.76116100e-01, 4.55860524e-01, 4.14612424e-28, ...,
         2.99735254e-28, 3.16581418e-27, 1.80759041e-28],
        [2.70915463e-01, 5.29590772e-02, 7.92932795e-03, ...,
         4.69370336e-03, 4.37640715e-03, 2.72160536e-19],
        [5.95177729e-01, 6.11959885e-02, 2.33560303e-02, ...,
         5.33144951e-03, 4.94030911e-03, 5.87007440e-03],
        ...,
        [1.26302796e-01, 2.57815729e-02, 5.59216197e-01, ...,
         2.34197724e-03, 2.12847745e-03, 2.61303089e-03],
        [1.87561440e-72, 3.79521338e-89, 2.14280220e-92, ...,
         3.92358963e-92, 2.09131616e-90, 6.05531557e-94],
        [9.29531770e-01, 7.44133660e-10, 5.23586064e-12, ...,
         2.56166942e-12, 1.17947524e-11, 1.67299045e-12]]))

## 출력 결과

In [100]:
# 테스트 세트 추론 - kobertopic 결과 사용
test_predictions = []

# kobertopic_val_topics 리스트에서 각 문서의 토픽을 예측 결과로 사용
for topic in kobertopic_val_topics:
    if topic == -1:
        test_predictions.append(56)  # -1일 경우 56번으로 대체
    else:
        test_predictions.append(topic)

# 라벨 디코딩 - 56번은 "기타"로 설정
label_decoder = {i: label for label, i in label_encoder.items()}
label_decoder[56] = "기타"
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

# 출력 예시
decoded_predictions[:5]

['기타', '기타', '문화:전시_공연', '기타', '기타']

In [101]:
from sklearn.metrics import f1_score

val_f1 = f1_score(kobertopic_val_topics, val_sample_10000_labels, average='macro')
print(f"Validation F1 Score: {val_f1:.2f}")

Validation F1 Score: 0.00


In [102]:
val_sample_10000_labels

,label
6252,1
4684,1
1731,1
4742,7
4521,0
...,...
6412,1
8285,11
7853,0
1095,1


In [111]:
kobertopic_val_topics_df = pd.DataFrame(kobertopic_val_topics, columns=["Topic"])
kobertopic_val_topics_df

,Topic
0,-1
1,-1
2,0
3,-1
4,-1
...,...
1995,-1
1996,-1
1997,2
1998,-1


In [116]:
kobertopic_val_topics_df.Topic.unique()

array([-1,  0, 29, 33, 28,  7, 11, 14,  3,  2, 16,  5, 12,  9,  1,  4, 23,
       19, 27, 13,  6, 32, 25, 21, 17, 35, 31, 41, 26, 43, 20, 36,  8, 18,
       34, 10, 22, 15, 39, 24, 37, 47, 30, 44])

In [118]:
# 라벨 디코딩 - 56번은 "기타"로 설정
label_decoder = {i: label for label, i in label_encoder.items()}
label_decoder[56] = "기타"
val_sample_10000_class = [label_decoder[pred] for pred in val_sample_10000_labels]

val_f1 = f1_score(decoded_predictions, val_sample_10000_class, average='macro')
print(f"Validation F1 Score: {val_f1:.2f}")

Validation F1 Score: 0.00


## KoBERTopic2

In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import umap
import hdbscan
import re

# TF-IDF를 이용한 벡터화
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)

# 차원 축소: UMAP의 파라미터 조정
dumap_model = umap.UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# 클러스터링: HDBSCAN의 min_cluster_size와 min_samples 조정
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=15, min_samples=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# BERTopic 모델 학습
kobertopic2 = BERTopic(embedding_model=embedding_model,
                      vectorizer_model=vectorizer,
                      umap_model=dumap_model,
                      hdbscan_model=hdbscan_model,
                      nr_topics=56,
                      top_n_words=10,
                      calculate_probabilities=True,
                      verbose=True)

In [125]:
kobertopic2_train_topics, kobertopic2_train_probs = kobertopic2.fit_transform(train_sample_10000_docs['text'], y=train_sample_10000_labels)

2024-10-07 08:26:52,550 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/250 [00:00<?, ?it/s]

2024-10-07 08:27:21,691 - BERTopic - Embedding - Completed ✓
2024-10-07 08:27:21,693 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-07 08:27:30,561 - BERTopic - Dimensionality - Completed ✓
2024-10-07 08:27:30,563 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-07 08:27:37,777 - BERTopic - Cluster - Completed ✓
2024-10-07 08:27:37,779 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-07 08:27:44,211 - BERTopic - Representation - Completed ✓
2024-10-07 08:27:44,213 - BERTopic - Topic reduction - Reducing number of topics
2024-10-07 08:27:49,886 - BERTopic - Topic reduction - Reduced number of topics from 123 to 56


In [126]:
kobertopic2_train_topics[:5], kobertopic2_train_probs

([-1, 0, 0, -1, 0],
 array([[8.69791865e-002, 2.05321617e-001, 4.77405608e-003, ...,
         1.64236987e-003, 4.15113012e-003, 2.67951592e-003],
        [7.41280499e-001, 5.50849968e-002, 2.99597326e-003, ...,
         1.69869877e-003, 7.07320650e-003, 2.39331530e-003],
        [1.00000000e+000, 1.17312539e-306, 7.14925282e-308, ...,
         5.58912348e-308, 1.54877345e-307, 5.97208510e-308],
        ...,
        [7.82524034e-001, 3.39336952e-002, 2.17143462e-003, ...,
         1.78626649e-003, 4.20782351e-003, 1.85165643e-003],
        [3.50104510e-306, 1.98852223e-306, 3.43765891e-307, ...,
         2.69604504e-307, 1.06547918e-307, 4.44536983e-307],
        [1.00000000e+000, 9.62995871e-307, 6.52419088e-308, ...,
         4.64984340e-308, 1.03791877e-307, 5.62352290e-308]]))

In [127]:
kobertopic2_val_topics, kobertopic2_val_probs = kobertopic2.transform(val_sample_10000_docs['text'].tolist())

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2024-10-07 08:28:00,297 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2024-10-07 08:28:05,638 - BERTopic - Dimensionality - Completed ✓
2024-10-07 08:28:05,639 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-10-07 08:28:05,807 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2024-10-07 08:28:07,985 - BERTopic - Probabilities - Completed ✓
2024-10-07 08:28:07,986 - BERTopic - Cluster - Completed ✓


In [128]:
kobertopic2_val_topics[:5], kobertopic2_val_probs

([-1, -1, 0, -1, -1],
 array([[1.80753078e-01, 3.94565013e-13, 2.64898228e-14, ...,
         3.13653160e-16, 1.48816325e-15, 2.30977716e-14],
        [2.44963425e-01, 3.65364605e-02, 3.15648774e-03, ...,
         8.92453684e-39, 3.80047371e-37, 2.94092616e-03],
        [2.36207087e-01, 2.57404174e-02, 1.98100775e-03, ...,
         1.61385316e-03, 2.70843203e-03, 1.82611329e-03],
        ...,
        [1.89513011e-03, 3.96759800e-04, 3.17076358e-05, ...,
         3.08309508e-05, 4.18074689e-05, 2.97535955e-05],
        [5.39581654e-01, 3.63878170e-02, 3.14043862e-03, ...,
         2.83229108e-05, 5.71875721e-05, 2.92436370e-03],
        [9.59778494e-01, 8.52127728e-19, 7.23757753e-20, ...,
         6.48072355e-22, 1.36640322e-21, 6.69665715e-20]]))

## 출력 결과

In [129]:
from sklearn.metrics import f1_score

val_f1 = f1_score(kobertopic2_val_topics, val_sample_10000_labels, average='macro')
print(f"Validation F1 Score: {val_f1:.2f}")

Validation F1 Score: 0.01


## KoBERTopic3 (하이퍼파라미터 튜닝)

In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import umap
import hdbscan
import pandas as pd
from tqdm import tqdm
import optuna  # 하이퍼 파라미터 튜닝을 위한 라이브러리
import re
import numpy as np
from sklearn.metrics import accuracy_score

train_sample_10000_docs_list = train_sample_10000_docs['text'].tolist()

# 데이터 전처리 개선: 특수문자 제거 및 전처리 강화
preprocessed_documents = [
    re.sub(r'[^\w\s]', '', line).strip() for line in train_sample_10000_docs_list
    if line and not line.replace(' ', '').isdecimal()
]

In [142]:
# 하이퍼 파라미터 튜닝 함수 정의
def objective(trial):
    # 하이퍼 파라미터 샘플링
    max_features = trial.suggest_int("max_features", 500, 3000)
    n_neighbors = trial.suggest_int("n_neighbors", 5, 50)
    min_dist = trial.suggest_float("min_dist", 0.0, 1.0)
    top_n_words = trial.suggest_int("top_n_words", 5, 20)

    # TF-IDF를 이용한 벡터화
    vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=max_features)

    # 차원 축소: UMAP의 파라미터 조정
    umap_model = umap.UMAP(n_neighbors=n_neighbors, n_components=5, min_dist=min_dist, metric='cosine')

    # K-means 클러스터링 사용
    kmeans_model = KMeans(n_clusters=56, random_state=42)

    # BERTopic 모델 학습
    kobertopic3 = BERTopic(
        embedding_model=embedding_model,
        vectorizer_model=vectorizer,
        umap_model=umap_model,
        hdbscan_model=kmeans_model,
        nr_topics=56,
        top_n_words=top_n_words,
        calculate_probabilities=True,
        verbose=False  # 튜닝 시에는 로그를 줄이기 위해 False로 설정
    )

    # 문서에 대해 모델 학습 수행
    kobertopic3_train_topics_hp, _ = kobertopic3.fit_transform(preprocessed_documents, y=train_sample_10000_labels)

    # 정확도 계산
    accuracy = accuracy_score(train_sample_10000_labels.to_list(), np.array(kobertopic3_train_topics_hp))
    return accuracy

# Optuna를 사용한 하이퍼 파라미터 튜닝
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 최적의 하이퍼 파라미터 출력
print("Best hyperparameters:", study.best_params)

[I 2024-10-07 09:14:39,212] A new study created in memory with name: no-name-266f61e9-1883-48dd-921c-2543caa61161
[I 2024-10-07 09:15:37,382] Trial 0 finished with value: 0.01075 and parameters: {'max_features': 858, 'n_neighbors': 30, 'min_dist': 0.7537089115062251, 'top_n_words': 12}. Best is trial 0 with value: 0.01075.
[I 2024-10-07 09:16:30,997] Trial 1 finished with value: 0.018 and parameters: {'max_features': 2851, 'n_neighbors': 10, 'min_dist': 0.8887120024817686, 'top_n_words': 10}. Best is trial 1 with value: 0.018.
[I 2024-10-07 09:17:29,078] Trial 2 finished with value: 0.006875 and parameters: {'max_features': 1937, 'n_neighbors': 45, 'min_dist': 0.643874050336864, 'top_n_words': 8}. Best is trial 1 with value: 0.018.
[I 2024-10-07 09:18:26,331] Trial 3 finished with value: 0.0065 and parameters: {'max_features': 589, 'n_neighbors': 43, 'min_dist': 0.5053625238002677, 'top_n_words': 11}. Best is trial 1 with value: 0.018.
[I 2024-10-07 09:19:20,916] Trial 4 finished with 

Best hyperparameters: {'max_features': 2636, 'n_neighbors': 16, 'min_dist': 0.2845570581724549, 'top_n_words': 6}


In [144]:
# 최적의 하이퍼 파라미터로 최종 모델 학습
best_params = study.best_params
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)
umap_model = umap.UMAP(n_neighbors=best_params["n_neighbors"], n_components=5, min_dist=best_params["min_dist"], metric='cosine')
kmeans_model = KMeans(n_clusters=56, random_state=42)

kobertopic3 = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer,
    umap_model=umap_model,
    hdbscan_model=kmeans_model,
    nr_topics=56,
    top_n_words=best_params["top_n_words"],
    calculate_probabilities=True,
    verbose=True
)

kobertopic3_train_topics, kobertopic3_train_probs = kobertopic3.fit_transform(preprocessed_documents, y=train_sample_10000_labels)

# 정확도 계산
accuracy = accuracy_score(train_sample_10000_labels.to_list(), np.array(kobertopic3_train_topics))
print(f"모델의 정확도: {accuracy:.2f}")

# 토픽 예측을 위한 새로운 데이터셋 적용
val_sample_10000_docs_list = val_sample_10000_docs['text'].tolist()
kobertopic3_val_topics, kobertopic_val3_probs = kobertopic3.transform(val_sample_10000_docs_list)

# 평가지표 계산
val_sample_10000_labels = val_sample_10000_docs['label'].tolist()
val_f1 = f1_score(kobertopic3_val_topics, val_sample_10000_labels, average='macro')
print(f"Validation F1 Score: {val_f1:.2f}")

2024-10-07 09:39:41,211 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/250 [00:00<?, ?it/s]

2024-10-07 09:40:09,970 - BERTopic - Embedding - Completed ✓
2024-10-07 09:40:09,971 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-10-07 09:40:21,664 - BERTopic - Dimensionality - Completed ✓
2024-10-07 09:40:21,666 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-10-07 09:40:21,752 - BERTopic - Cluster - Completed ✓
2024-10-07 09:40:21,753 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-10-07 09:40:30,646 - BERTopic - Representation - Completed ✓
2024-10-07 09:40:30,647 - BERTopic - Topic reduction - Reducing number of topics
2024-10-07 09:40:30,651 - BERTopic - Topic reduction - Reduced number of topics from 56 to 56


모델의 정확도: 0.02


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2024-10-07 09:40:45,638 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2024-10-07 09:40:50,655 - BERTopic - Dimensionality - Completed ✓
2024-10-07 09:40:50,656 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-10-07 09:40:50,659 - BERTopic - Cluster - Completed ✓


Validation F1 Score: 0.01


In [150]:
# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in kobertopic3_val_topics]

# 결과를 데이터프레임으로 변환
result = pd.DataFrame({
    'Actual': val_sample_10000_docs['분류'],
    'Predicted': decoded_predictions
})

# 결과 확인
result

,Actual,Predicted
6252,지역,정치:행정_자치
4684,지역,문화:생활
1731,지역,사회:미디어
4742,정치:국회_정당,경제:증권_증시
4521,문화:전시_공연,사회:날씨
...,...,...
6412,지역,경제:산업_기업
8285,사회:교육_시험,경제:반도체
7853,문화:전시_공연,문화:전시_공연
1095,지역,문화:방송_연예
